In [ ]:
from __future__ import division, print_function
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import glob


## preparing training data
only one day...

In [8]:
!wget -q -r -nH -np --cut-dirs=2 http://people.phys.ethz.ch/~ast/cosmo/bgs_example_data/

In [9]:
!mkdir -p bgs_example_data/seek_cache

In [10]:
!seek --file-prefix='./bgs_example_data' --post-processing-prefix='bgs_example_data/seek_cache' --chi-1=20 --overwrite=True seek.config.process_survey_fft

/bin/sh: 1: seek: not found


## setting up the unet

In [11]:
from scripts.radio_util import DataProvider
from tf_unet import unet


In [12]:
files = glob.glob('bgs_example_data/seek_cache/*')

In [13]:
data_provider = DataProvider(600, files)

net = unet.Unet(channels=data_provider.channels, 
                n_class=data_provider.n_class, 
                layers=3, 
                features_root=64,
                add_regularizers=True,
                )


AssertionError: No training files

## training the network
only one epoch. For good results many more are neccessary

In [ ]:

trainer = unet.Trainer(net, optimizer="momentum", opt_kwargs=dict(momentum=0.2))
path = trainer.train(data_provider, "./unet_trained_bgs_example_data", 
                     training_iters=32, 
                     epochs=1, 
                     dropout=0.5, 
                     display_step=2)



## running the prediction on the trained unet

In [ ]:
data_provider = DataProvider(12000, files)
x_test, y_test = data_provider(1)
prediction = net.predict(path, x_test)

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(12,4))
ax[0].imshow(x_test[0,...,0], aspect="auto")
ax[1].imshow(y_test[0,...,1], aspect="auto")
ax[2].imshow(prediction[0,...,1], aspect="auto")